# Digit Recognizer

In [1]:
import pandas as pd 
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt
import warnings

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow 

import scipy
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


# Set the seed value for experiment reproduci.bility.
seed = 1842
tensorflow.random.set_seed(seed)
np.random.seed(seed)
# Turn off warnings for cleaner looking notebook
warnings.simplefilter('ignore')


df=pd.read_csv('train.csv')

df


Num GPUs Available:  1


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
X=df.drop('label',axis=1)
Y=df.label

In [3]:
Y=pd.get_dummies(Y,prefix='Num')

In [4]:
X.describe()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [5]:
X=X/255

X=X.values.reshape(-1,28,28)

In [6]:
X.shape

(42000, 28, 28)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)

In [8]:
validation_data=X_val,y_val

# ANN

İlk önce kendimize CNN kullanmadan bir mimari tasarlıyoruz.

Buradaki ayarlar hypertuning ile karşılaşırılabilir. Ben sadece iki çeşit denedim.

In [16]:
model = keras.models.Sequential([
keras.layers.Flatten(input_shape = (28,28,1)),                             
keras.layers.Dense(300, activation = 'relu' ),

keras.layers.Dense(400, activation = 'relu' ),

keras.layers.Dense(400, activation = 'relu' ),

keras.layers.Dense(400, activation = 'relu' ),
keras.layers.Dense(10, activation = 'softmax')])


model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)

model.fit(X_train,y_train, epochs=20, validation_data=(validation_data), callbacks=callback)
#save model
#data manipulation 
#include graphs 

Epoch 1/20
880/880 [==============================] - 4s 4ms/step - loss: 0.2882 - auc: 0.9929 - val_loss: 0.1649 - val_auc: 0.9971
Epoch 2/20
880/880 [==============================] - 4s 4ms/step - loss: 0.1296 - auc: 0.9975 - val_loss: 0.1444 - val_auc: 0.9968
Epoch 3/20
880/880 [==============================] - 3s 4ms/step - loss: 0.0933 - auc: 0.9983 - val_loss: 0.1124 - val_auc: 0.9978
Epoch 4/20
880/880 [==============================] - 3s 4ms/step - loss: 0.0701 - auc: 0.9988 - val_loss: 0.1033 - val_auc: 0.9973
Epoch 5/20
880/880 [==============================] - 3s 4ms/step - loss: 0.0637 - auc: 0.9990 - val_loss: 0.1609 - val_auc: 0.9948
Epoch 6/20
880/880 [==============================] - 3s 4ms/step - loss: 0.0537 - auc: 0.9991 - val_loss: 0.1433 - val_auc: 0.9956
Epoch 7/20
880/880 [==============================] - 3s 4ms/step - loss: 0.0448 - auc: 0.9993 - val_loss: 0.1380 - val_auc: 0.9957


In [21]:
loss, accuracy = model.evaluate(X_val,y_val)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

434/434 [==============================] - 1s 3ms/step - loss: 0.1033 - auc: 0.9973
Loss:  0.10326448082923889
Accuracy:  0.9973464608192444


In [11]:
#simple ann with 3 layers 
model = keras.models.Sequential([
keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
keras.layers.experimental.preprocessing.RandomContrast(0.5),    

keras.layers.Flatten(input_shape = (28,28,1)),                             
keras.layers.Dense(300, activation = 'relu' ),
keras.layers.Dropout(0.1),
keras.layers.Dense(400, activation = 'relu' ),
keras.layers.Dropout(0.1),
keras.layers.Dense(400, activation = 'relu' ),
keras.layers.Dropout(0.1),
keras.layers.Dense(400, activation = 'relu' ),
keras.layers.Dense(10, activation = 'softmax')])


model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)



In [8]:
model.fit(X_train,y_train, epochs=20, validation_data=(validation_data), callbacks=callback)
#save model
#data manipulation 
#include graphs 

Epoch 1/20
1312/1312 [==============================] - 48s 35ms/step - loss: 0.6686 - accuracy: 0.9694 - val_loss: 0.1373 - val_accuracy: 0.9714
Epoch 2/20
1312/1312 [==============================] - 48s 37ms/step - loss: 0.1581 - accuracy: 0.9713 - val_loss: 0.1741 - val_accuracy: 0.9714
Epoch 3/20
1312/1312 [==============================] - 49s 37ms/step - loss: 0.1807 - accuracy: 0.9713 - val_loss: 0.1454 - val_accuracy: 0.9714
Epoch 4/20
1312/1312 [==============================] - 53s 41ms/step - loss: 0.1398 - accuracy: 0.9714 - val_loss: 0.1339 - val_accuracy: 0.9714
Epoch 5/20
1312/1312 [==============================] - 51s 39ms/step - loss: 0.1380 - accuracy: 0.9714 - val_loss: 0.1326 - val_accuracy: 0.9714
Epoch 6/20
1312/1312 [==============================] - 51s 39ms/step - loss: 0.1332 - accuracy: 0.9714 - val_loss: 0.1309 - val_accuracy: 0.9714
Epoch 7/20
1312/1312 [==============================] - 51s 39ms/step - loss: 0.1868 - accuracy: 0.9713 - val_loss: 0.1325 -

In [12]:
loss, accuracy = model.evaluate((validation_data))
print("Loss: ", loss)
print("Accuracy: ", accuracy)

328/328 [==============================] - 6s 17ms/step - loss: 0.1304 - accuracy: 0.9714
Loss:  0.13037563860416412
Accuracy:  0.9714067578315735


# CNN

### First CNN

Burada konvolüsyon işlemi yaparak yapay sinir ağımıza farklı bir yaklaşım sergilettiriyoruz.

Üstteki gibi burada da filtrelerle oynamalar yapıp doğruluk oranının değişimine baktım.

In [22]:
#experiment with Convolutional neural net
model = keras.models.Sequential([
keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = (28,28,1)),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.Flatten(),
keras.layers.Dense(100, activation='relu'),
keras.layers.Dense(10, activation ='softmax')])

In [23]:
model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)

In [24]:
model.fit(X_train,y_train, epochs=20, validation_data=validation_data, callbacks=callback)
#save model
#early stopping

Epoch 1/20
880/880 [==============================] - 9s 5ms/step - loss: 0.2113 - auc: 0.9963 - val_loss: 0.0716 - val_auc: 0.9990
Epoch 2/20
880/880 [==============================] - 4s 5ms/step - loss: 0.0586 - auc: 0.9993 - val_loss: 0.0588 - val_auc: 0.9991
Epoch 3/20
880/880 [==============================] - 4s 5ms/step - loss: 0.0429 - auc: 0.9993 - val_loss: 0.0662 - val_auc: 0.9990
Epoch 4/20
880/880 [==============================] - 4s 4ms/step - loss: 0.0315 - auc: 0.9996 - val_loss: 0.0350 - val_auc: 0.9993
Epoch 5/20
880/880 [==============================] - 4s 4ms/step - loss: 0.0243 - auc: 0.9996 - val_loss: 0.0414 - val_auc: 0.9992
Epoch 6/20
880/880 [==============================] - 4s 5ms/step - loss: 0.0217 - auc: 0.9998 - val_loss: 0.0422 - val_auc: 0.9993
Epoch 7/20
880/880 [==============================] - 5s 5ms/step - loss: 0.0176 - auc: 0.9998 - val_loss: 0.0408 - val_auc: 0.9990


In [26]:
loss, accuracy = model.evaluate(X_val,y_val)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

434/434 [==============================] - 1s 3ms/step - loss: 0.0350 - auc: 0.9993
Loss:  0.03500228747725487
Accuracy:  0.9993278384208679


### Second CNN

In [27]:
model = keras.models.Sequential([
keras.layers.Conv2D(224, (3, 3), activation='relu', input_shape = (28,28,1)),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(448, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(448, (2, 2), activation='relu'),
keras.layers.Flatten(),
keras.layers.Dense(100, activation='relu'),
keras.layers.Dense(10, activation ='softmax')])

In [28]:
model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)

In [29]:
model.fit(X_train,y_train, epochs=20, validation_data=(validation_data), callbacks=callback)
#save model
#early stopping

Epoch 1/20
880/880 [==============================] - 9s 9ms/step - loss: 0.1610 - auc: 0.9971 - val_loss: 0.0558 - val_auc: 0.9993
Epoch 2/20
880/880 [==============================] - 8s 9ms/step - loss: 0.0503 - auc: 0.9992 - val_loss: 0.0563 - val_auc: 0.9990
Epoch 3/20
880/880 [==============================] - 8s 9ms/step - loss: 0.0384 - auc: 0.9993 - val_loss: 0.0528 - val_auc: 0.9990
Epoch 4/20
880/880 [==============================] - 8s 9ms/step - loss: 0.0262 - auc: 0.9997 - val_loss: 0.0416 - val_auc: 0.9987
Epoch 5/20
880/880 [==============================] - 8s 9ms/step - loss: 0.0227 - auc: 0.9998 - val_loss: 0.0397 - val_auc: 0.9991
Epoch 6/20
880/880 [==============================] - 8s 9ms/step - loss: 0.0204 - auc: 0.9996 - val_loss: 0.0507 - val_auc: 0.9987
Epoch 7/20
880/880 [==============================] - 8s 9ms/step - loss: 0.0144 - auc: 0.9998 - val_loss: 0.0514 - val_auc: 0.9983
Epoch 8/20
880/880 [==============================] - 8s 9ms/step - loss: 0.

In [31]:
loss, accuracy = model.evaluate(X_val,y_val)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

434/434 [==============================] - 2s 4ms/step - loss: 0.0397 - auc: 0.9991
Loss:  0.03973952308297157
Accuracy:  0.9990682005882263


## Best Model

In [33]:
#experiment with Convolutional neural net
model = keras.models.Sequential([
keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = (28,28,1)),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.Flatten(),
keras.layers.Dense(100, activation='relu'),
keras.layers.Dense(10, activation ='softmax')])


model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)

In [34]:
model.fit(X_train,y_train, epochs=20, validation_data=validation_data, callbacks=callback)
#save model
#early stopping

Epoch 1/20
880/880 [==============================] - 5s 5ms/step - loss: 0.1945 - auc: 0.9969 - val_loss: 0.0688 - val_auc: 0.9993
Epoch 2/20
880/880 [==============================] - 4s 5ms/step - loss: 0.0542 - auc: 0.9993 - val_loss: 0.0614 - val_auc: 0.9990
Epoch 3/20
880/880 [==============================] - 4s 5ms/step - loss: 0.0416 - auc: 0.9993 - val_loss: 0.0510 - val_auc: 0.9991
Epoch 4/20
880/880 [==============================] - 4s 5ms/step - loss: 0.0305 - auc: 0.9997 - val_loss: 0.0426 - val_auc: 0.9992
Epoch 5/20
880/880 [==============================] - 5s 5ms/step - loss: 0.0246 - auc: 0.9997 - val_loss: 0.0460 - val_auc: 0.9992
Epoch 6/20
880/880 [==============================] - 4s 5ms/step - loss: 0.0199 - auc: 0.9997 - val_loss: 0.0412 - val_auc: 0.9993
Epoch 7/20
880/880 [==============================] - 4s 5ms/step - loss: 0.0150 - auc: 0.9998 - val_loss: 0.0445 - val_auc: 0.9989
Epoch 8/20
880/880 [==============================] - 4s 5ms/step - loss: 0.

In [57]:
df_test=pd.read_csv('test.csv')
df_test=df_test/255

df_test=df_test.values.reshape(-1,28,28)

In [58]:
y_pred=model.predict(df_test)

875/875 [==============================] - 1s 1ms/step


In [63]:
y_pred=pd.DataFrame(np.argmax(y_pred,axis=1),columns=['Label'])

In [64]:
submission = pd.read_csv("sample_submission.csv")
submission["Label"] = y_pred
submission.to_csv('submission.csv', index=False)

In [65]:
!kaggle competitions submit -c digit-recognizer -f submission.csv -m ""

Successfully submitted to Digit Recognizer



  0%|          | 0.00/235k [00:00<?, ?B/s]
 88%|████████▊ | 208k/235k [00:00<00:00, 909kB/s]
100%|██████████| 235k/235k [00:01<00:00, 143kB/s]
